In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

from keras.layers import Concatenate, Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import  Adam
import keras.backend as k

# import matplotlib.pyplot as plt
# %matplotlib inline
# from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
# pd.options.display.precision = 15
from category_encoders.cat_boost import CatBoostEncoder

# import lightgbm as lgb
# import xgboost as xgb
# import time
# import datetime
# from catboost import CatBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
# from sklearn import metrics
# from sklearn import linear_model
import gc
import pickle
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt

# import networkx as nx
# import matplotlib.pyplot as plt

gc.collect()
# alt.renderers.enable('notebook')

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

0

In [5]:
main_path = r'../..'

In [6]:
import sys
sys.path.append(main_path)
from BayDS import *

In [7]:
experiment_name = '31.08'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [9]:
#start here
data_folder = main_path+'/Data'
y = pd.read_pickle(f'{data_folder}/y.pkl')
X = pd.read_pickle(f'{data_folder}/X_encoded_scaled.pkl').astype(np.float32)
test = pd.read_pickle(f'{data_folder}/test_encoded_scaled.pkl').astype(np.float32)
gc.collect()

35

In [10]:
# Setting model_folder
model_name = 'keras-4'
model_folder = f'{main_learning_folder}/{model_name}'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [11]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)
# folds = GroupKFold(n_splits=5)
# groups = pd.read_pickle('./groups.pkl').values

In [12]:
y.value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [13]:
def NNModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[X.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits
    x = Dense(400, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(400, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(200, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = BatchNormalization()(x)    
    
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 8192,
    'epochs': 60,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [14]:
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)


In [15]:
NNModel_maker().save(f'{model_folder}/keras.mdl')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9394828649611977625, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1452988825
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4512666971317458215
 physical_device_desc: "device: 0, name: Quadro M1000M, pci bus id: 0000:01:00.0, compute capability: 5.0"]

In [17]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [18]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [19]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU':8},log_device_placement=True) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [20]:
gc.collect()
result_dict_keras = train_model_classification(model=NNModel_maker, 
                                             X=X,
                                             X_test=test,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Fri Sep  6 23:11:20 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/60
472432/472432 [==============================] - 22s 46us/step - loss: 0.6667 - val_loss: 0.8111
Epoch 2/60
472432/472432 [==============================] - 19s 41us/step - loss: 0.4309 - val_loss: 0.5337
Epoch 3/60
472432/472432 [==============================] - 19s 40us/step - loss: 0.2878 - val_loss: 0.4075
Epoch 4/60
472432/472432 [==============================] - 19s 39us/step - loss: 0.2118 - val_loss: 0.3171
Epoch 5/60
472432/472432 [==============================] - 19s 40us/step - loss: 0.1689 - val_loss: 0.2644
Epoch 6/60
472432/472432 [==============================] - 19s 40us/step - loss: 0.1469 - val_loss: 0.2036
Epoch 7/60
472432/472432 [==============================] - 19s 40us/step - loss: 0.1335 - val_loss: 0.1681
Epoch 8/60
472432/472432 [==============================] - 19s 39us/step - loss: 0.1249 - val_loss: 0.1480
Epoch 9/60
472432/472432 [===============

Epoch 14/60
472432/472432 [==============================] - 18s 38us/step - loss: 0.0930 - val_loss: 0.0963
Epoch 15/60
472432/472432 [==============================] - 18s 38us/step - loss: 0.0903 - val_loss: 0.0948
Epoch 16/60
472432/472432 [==============================] - 18s 38us/step - loss: 0.0878 - val_loss: 0.0904
Epoch 17/60
472432/472432 [==============================] - 18s 38us/step - loss: 0.0866 - val_loss: 0.0916
Epoch 18/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0848 - val_loss: 0.0909
Epoch 19/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0833 - val_loss: 0.0897
Epoch 20/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0820 - val_loss: 0.0900
Epoch 21/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0810 - val_loss: 0.0869
Epoch 22/60
472432/472432 [==============================] - 18s 38us/step - loss: 0.0796 - val_loss: 0.0869
Epoch 23/60
472432/

472432/472432 [==============================] - 20s 41us/step - loss: 0.0703 - val_loss: 0.0816
Epoch 41/60
472432/472432 [==============================] - 19s 40us/step - loss: 0.0698 - val_loss: 0.0804
Epoch 42/60
472432/472432 [==============================] - 19s 39us/step - loss: 0.0698 - val_loss: 0.0824
Epoch 43/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0694 - val_loss: 0.0812
Epoch 44/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0691 - val_loss: 0.0814
Epoch 45/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0681 - val_loss: 0.0806
Epoch 46/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0680 - val_loss: 0.0806
Epoch 47/60
472432/472432 [==============================] - 19s 40us/step - loss: 0.0674 - val_loss: 0.0795
Epoch 48/60
472432/472432 [==============================] - 18s 39us/step - loss: 0.0678 - val_loss: 0.0816
Epoch 49/60
472432/472432 [====

In [21]:
sub = pd.read_csv(f'../../data/sample_submission.csv')

In [22]:
sub['isFraud'] = result_dict_keras['prediction']
sub.to_csv(f'{model_folder}/ieee_nn.csv', index=False)

In [23]:
result_dict_keras['prediction'].shape

(506691, 1)

In [24]:
test.shape

(506691, 789)

In [25]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_keras,f)
#     f.write(q)